In [4]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle

cap = cv2.VideoCapture(0)

Counter_LowerHand1 = 0
Counter_LowerHand2 = 0
Counter_TopHand = 0
Counter_Block = 0
Stage_Lower = None
Stage_Top = None
Stage_Block = None

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # 右手角度
            RIGHT_SHOULDER = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            RIGHT_ELBOW = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            RIGHT_WRIST = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            # 左手角度
            LEFT_SHOULDER = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            LEFT_ELBOW = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            LEFT_WRIST = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            # 右肩角度
            LEFT_SHOULDER = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            RIGHT_SHOULDER = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            RIGHT_ELBOW = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]

            # 左肩角度
            RIGHT_SHOULDER = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            LEFT_SHOULDER = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            LEFT_ELBOW = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            
            # 右腳角度
            RIGHT_HIP = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            
            RIGHT_KNEE = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            
            RIGHT_ANKLE = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            
            #左腳角度
            LEFT_HIP = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            
            LEFT_KNEE = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            
            LEFT_ANKLE = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            
            
            # 右膝角度
            angleRightLeg = round(calculate_angle(RIGHT_HIP, RIGHT_KNEE, RIGHT_ANKLE),0)
            # 左膝角度
            angleLeftLeg = round(calculate_angle(LEFT_HIP, LEFT_KNEE, LEFT_ANKLE),0)
            # 右手角度
            angleRightArm = round(calculate_angle(RIGHT_SHOULDER, RIGHT_ELBOW, RIGHT_WRIST),0)
            # 左手角度
            angleLeftArm = round(calculate_angle(LEFT_SHOULDER, LEFT_ELBOW, LEFT_WRIST),0)
            # 右肩角度
            angleRightshoulder = round(calculate_angle(LEFT_SHOULDER, RIGHT_SHOULDER, RIGHT_ELBOW),0)
            # 左肩角度
            angleLeftshoulder = round(calculate_angle(RIGHT_SHOULDER, LEFT_SHOULDER, LEFT_ELBOW),0)
            
            
            
            
            
            # 低手角度、肩膀、手肘的角度調整，如果在區間內，則開始計分
            if angleRightshoulder < 100 and angleRightshoulder > 65 and angleLeftshoulder < 100 and angleLeftshoulder > 65:
                if angleRightArm < 180 and angleRightArm > 160 and angleLeftArm < 180 and angleLeftArm > 160 :
                    if angleRightLeg < 160:
                        Stage_Lower = "downRight"
                    if angleRightLeg > 170 and Stage_Lower =='downRight':
                        Stage_Lower="upRight"
                        Counter_LowerHand1 += 1
                    if angleLeftLeg < 160:
                        Stage_Lower = "downLeft"
                    if angleLeftLeg > 170 and Stage_Lower =='downLeft':
                        Stage_Lower="upLeft"
                        Counter_LowerHand2 += 1
            
            # 上手角度、肩膀、手肘的角度調整，如果在區間內，則開始計分
            if angleRightshoulder > 150 and angleRightshoulder < 180 and \
                    angleLeftshoulder > 150 and angleRightshoulder < 180 and \
                    angleRightArm > 40 and angleRightArm < 80 and \
                    angleLeftArm > 40 and angleLeftArm < 80:
                Stage_Top = "TopDown"

            if angleRightshoulder > 100 and angleRightshoulder < 125 and \
                    angleLeftshoulder > 100 and angleRightshoulder < 125 and \
                    angleRightArm > 130 and angleLeftArm > 130 and Stage_Top == 'TopDown':
                Stage_Top = "TopUp"
                Counter_TopHand += 1
                
            # 攔網角度、肩膀、手肘的角度調整，如果在區間內，則開始計分
            if angleRightshoulder > 90 and angleRightshoulder < 110 and \
                    angleLeftshoulder > 90 and angleRightshoulder < 110 and \
                    angleRightArm > 0 and angleRightArm < 20 and \
                    angleLeftArm > 0 and angleLeftArm < 20:
                Stage_Block = "BlockDown"

            if angleRightshoulder > 90 and angleRightshoulder < 140 and \
                    angleLeftshoulder > 90 and angleRightshoulder < 140 and \
                    angleRightArm > 160 and angleLeftArm > 160 and Stage_Block == 'BlockDown':
                Stage_Block = "BlockUp"
                Counter_Block += 1

        except:
            pass

        #分數框
        cv2.rectangle(image, (0, 0), (180, 40), (0, 219, 0), -1)

        #低手計分
        cv2.putText(image, str('Lower:'),
                    (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
        
        cv2.putText(image, str(Counter_LowerHand1 + Counter_LowerHand2),
                    (130, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
        
         # 各支點連一條線
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                  )
        
       
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    